In [8]:
import pandas as pd
import seaborn as sns
from Index_Calculations import facility_index_gen
import os
import glob
import numpy as np

## Import EIA-860 data

In [2]:
path = os.path.join('Data storage', 'EIA-860 2015', '2___Plant_Y2015.xlsx')
eia_860_plant = pd.read_excel(path, skiprows=1)

In [3]:
eia_860_plant.head()

,Utility ID,Utility Name,Plant Code,Plant Name,Street Address,City,State,Zip,County,Latitude,...,Ash Impoundment?,Ash Impoundment Lined?,Ash Impoundment Status,Transmission or Distribution System Owner,Transmission or Distribution System Owner ID,Transmission or Distribution System Owner State,Grid Voltage (kV),Grid Voltage 2 (kV),Grid Voltage 3 (kV),Natural Gas Pipeline Name
0,195,Alabama Power Co,2,Bankhead Dam,19001 Lock 17 Road,Northport,AL,35476,Tuscaloosa,33.4587,...,N,X,NaN,Alabama Power Co,195,AL,115,,,NaN
1,195,Alabama Power Co,3,Barry,North Highway 43,Bucks,AL,36512,Mobile,31.0069,...,Y,N,OP,Alabama Power Co,195,AL,230,,,Bay Gas
2,195,Alabama Power Co,4,Walter Bouldin Dam,750 Bouldin Dam Road,Wetumpka,AL,36092,Elmore,32.5839,...,N,X,NaN,Alabama Power Co,195,AL,115,,,NaN
3,195,Alabama Power Co,7,Gadsden,1000 Goodyear Ave,Gadsden,AL,35903,Etowah,34.0128,...,Y,N,OP,Alabama Power Co,195,AL,115,,,Southern Natural Gas
4,195,Alabama Power Co,8,Gorgas,460 Gorgas Road,Parrish,AL,35580,Walker,33.6443,...,Y,N,OP,Alabama Power Co,195,AL,230,,,NaN


In [4]:
eia_860_plant = eia_860_plant.loc[:, ['Plant Code', 'NERC Region']]

eia_860_plant.rename(columns={'NERC Region': 'region',
                              'Plant Code': 'plant id'}, inplace=True)

In [34]:
eia_860_plant.head()

,plant id,region
0,2,SERC
1,3,SERC
2,4,SERC
3,7,SERC
4,8,SERC


In [49]:
path = os.path.join('Data storage', 'Facility NERC labels.csv')
eia_860_plant.to_csv(path, index=False)

In [35]:
path = os.path.join('Data storage', 'Facility gen fuels and CO2 2017-05-25.zip')
df = pd.read_csv(path)
df.head()

,f,fuel,month,plant id,total fuel (mmbtu),year,generation (MWh),elec fuel (mmbtu),geography,last_updated,lat,lon,prime mover,datetime,quarter,all fuel fossil CO2 (kg),elec fuel fossil CO2 (kg),all fuel total CO2 (kg),elec fuel total CO2 (kg)
0,M,NG,3,10275,0.0,2017,0.0,0.0,USA-FL,2017-05-24T14:26:30-04:00,27.9114,-81.6006,ALL,2017-03-01,1,0.0,0.0,0.0,0.0
1,M,NG,2,10275,0.0,2017,0.0,0.0,USA-FL,2017-05-24T14:26:30-04:00,27.9114,-81.6006,ALL,2017-02-01,1,0.0,0.0,0.0,0.0
2,M,NG,1,10275,0.0,2017,0.0,0.0,USA-FL,2017-05-24T14:26:30-04:00,27.9114,-81.6006,ALL,2017-01-01,1,0.0,0.0,0.0,0.0
3,M,NG,12,10275,0.0,2016,0.0,0.0,USA-FL,2017-05-24T14:26:30-04:00,27.9114,-81.6006,ALL,2016-12-01,4,0.0,0.0,0.0,0.0
4,M,NG,11,10275,0.0,2016,0.0,0.0,USA-FL,2017-05-24T14:26:30-04:00,27.9114,-81.6006,ALL,2016-11-01,4,0.0,0.0,0.0,0.0


In [40]:
df = pd.merge(df, eia_860_plant, on='plant id', how='left')

In [48]:
for year in df['year'].unique():
    temp = df.loc[df['year'] == year]
    total_gen = temp['generation (MWh)'].sum()
    not_counted_gen = temp.loc[temp['region'].isnull(), 'generation (MWh)'].sum()
    percent_out = not_counted_gen / total_gen * 100
    print 'In {}, {:.1f}% of gen is missed'.format(year, percent_out) 

In 2017, 0.3% of gen is missed
In 2016, 0.0% of gen is missed
In 2015, 0.0% of gen is missed
In 2014, 0.2% of gen is missed
In 2013, 0.3% of gen is missed
In 2012, 0.4% of gen is missed
In 2011, 0.6% of gen is missed
In 2010, 0.7% of gen is missed
In 2009, 0.7% of gen is missed
In 2008, 0.8% of gen is missed
In 2007, 0.9% of gen is missed
In 2006, 0.9% of gen is missed
In 2005, 1.0% of gen is missed
In 2004, 1.1% of gen is missed
In 2003, 1.3% of gen is missed
In 2002, 1.4% of gen is missed
In 2001, 1.4% of gen is missed


## Calculate index values for each NERC region

In [4]:
facility_path = os.path.join('Data storage', 'Facility gen fuels and CO2 2017-05-25.zip')
epa_path = os.path.join('Data storage', 'Monthly EPA emissions 2017-05-25.csv')
ef_path = os.path.join('Data storage', 'Final emission factors.csv')
out_folder = os.path.join('Data storage', 'final NERC data')
NERC_path = os.path.join('Data storage', 'Facility NERC labels.csv')

for nerc in ['MRO', 'FRCC', 'NPCC', 'RFC',
             'SERC', 'WECC', 'ASCC', 'TRE', 'HICC']:
    facility_index_gen(facility_path=facility_path, 
                             epa_path=epa_path,
                             emission_factor_path=ef_path,
                             facility_info_path=NERC_path,
                             export_folder=out_folder, export_path_ext=' '
                             + nerc, region=nerc)

## Read data and create figures

In [5]:
path = os.path.join('Data storage', 'final NERC data', 'Monthly index*')
mi_fns = glob.glob(path)
path = os.path.join('Data storage', 'final NERC data', 'Monthly gen*')
mg_fns = glob.glob(path)

In [11]:
'MRO.csv'[:-4]

'MRO'

In [12]:
df_list = []
for f in mi_fns:
    region = f.split()[-1][:-4]
    df = pd.read_csv(f)
    df['region'] = region
    df_list.append(df)
full_mi = pd.concat(df_list)
full_mi.reset_index(inplace=True, drop=True)
full_mi.rename(columns={'index (g/kWh)': 'monthly index (g/kWh)'}, inplace=True)
full_mi['datetime'] = pd.to_datetime(full_mi['datetime'])

In [13]:
df_list = []
for f in mg_fns:
    region = f.split()[-1][:-4]
    df = pd.read_csv(f)
    df['region'] = region
    df_list.append(df)
full_mg = pd.concat(df_list)
full_mg.reset_index(inplace=True, drop=True)
full_mg['datetime'] = pd.to_datetime(full_mg['datetime'])

monthly_gen = pd.pivot_table(full_mg, index=['region', 'datetime'], 
                             values='generation (MWh)', columns='fuel category')
monthly_gen.reset_index(inplace=True, drop=False)
monthly_gen['Year'] = monthly_gen['datetime'].dt.year
monthly_gen.replace(np.nan, 0, inplace=True)

In [16]:
gen_index = pd.merge(monthly_gen, full_mi[['datetime', 'region', 'monthly index (g/kWh)']], 
                     on=['datetime', 'region'])
gen_index.head()

,region,datetime,Coal,Hydro,Natural Gas,Nuclear,Other,Other Renewables,Solar,Wind,Year,monthly index (g/kWh)
0,ASCC,2001-01-01,26493.302,103642.770,314206.726,0.0,64776.690,0.0,0.0,86.518,2001,545.186077
1,ASCC,2001-02-01,31263.704,89334.137,274760.266,0.0,60878.747,0.0,0.0,74.121,2001,588.848083
2,ASCC,2001-03-01,28459.894,98999.828,280795.473,0.0,84134.499,0.0,0.0,80.502,2001,596.381281
3,ASCC,2001-04-01,20455.924,83875.420,240608.882,0.0,71487.498,0.0,0.0,70.577,2001,565.183166
4,ASCC,2001-05-01,24096.734,84405.213,229902.421,0.0,70230.458,0.0,0.0,69.214,2001,563.503890


In [17]:
for region in gen_index['region'].unique():
    gen_index.loc[gen_index['region'] == region, 'Index variability'] = \
        gen_index.loc[gen_index['region']==region, 
                       'monthly index (g/kWh)'].rolling(window=12).std()
    
    gen_index.loc[gen_index['region'] == region, 
                   'Normalized Index variability'] = \
         gen_index.loc[gen_index['region']==region, 'Index variability'] / \
         gen_index.loc[gen_index['region']==region, 
                       'monthly index (g/kWh)'].rolling(window=12).mean()
gen_index.tail()

,region,datetime,Coal,Hydro,Natural Gas,Nuclear,Other,Other Renewables,Solar,Wind,Year,monthly index (g/kWh),Index variability,Normalized Index variability
1750,WECC,2016-11-01,1.352935e+07,10813903.0,1.275842e+07,5101804.0,310715.865,1229594.817,1510954.0,3440480.35,2016,397.483281,49.050524,0.126371
1751,WECC,2016-12-01,1.668025e+07,12705561.0,1.459834e+07,5328974.0,325077.814,1310444.734,1244865.0,4052181.30,2016,413.778458,49.065671,0.126401
1752,WECC,2017-01-01,1.634227e+07,12279167.0,1.500917e+07,5465776.0,415917.777,876279.171,508183.0,827359.00,2017,447.492806,50.590193,0.129865
1753,WECC,2017-02-01,1.306180e+07,10789414.0,1.031738e+07,4966951.0,403661.309,777001.518,574344.0,909178.00,2017,430.091742,51.712636,0.131267
1754,WECC,2017-03-01,1.164133e+07,13839887.0,9.028709e+06,5329821.0,433343.501,872926.744,1020723.0,1081541.00,2017,374.440973,46.425320,0.116447


In [18]:
base_year = 2005

In [19]:
fuels = ['Coal', 'Natural Gas', 'Other Renewables', 'Nuclear', 'Other',
         'Solar', 'Wind', 'Hydro']
gen_index['Total gen'] = gen_index.loc[:, fuels].sum(axis=1)
for fuel in fuels:
    # New columns that are being added
    col_percent = 'percent ' + fuel
    col_change = 'change in ' + fuel

    # Calculate percent of generation from each fuel type
    gen_index[col_percent] = gen_index.loc[:, fuel] / gen_index.loc[:, 'Total gen']

    # Percent of fuel in region in base year (entire year)
    for region in gen_index['region'].unique():
        percent_fuel_base = gen_index.loc[(gen_index['Year'] == base_year) & 
                                          (gen_index['region'] == region), fuel].sum() / gen_index.loc[(gen_index['Year'] == 2001) & 
                                                                                                     (gen_index['region'] == region), 'Total gen'].sum()

        # Use percent of fuel in 2001 to calculate change for each region/month
        gen_index.loc[gen_index['region'] == region, 
                      col_change] = (gen_index.loc[gen_index['region'] == region, col_percent] - percent_fuel_base) / percent_fuel_base
    
# Change in variability compared to average base year value
for region in gen_index['region'].unique():
    norm_variability_base = gen_index.loc[(gen_index['Year'] == base_year) & 
                                  (gen_index['region'] == region), 'Normalized Index variability'].mean()
    variability_base = gen_index.loc[(gen_index['Year'] == base_year) & 
                                  (gen_index['region'] == region), 'Index variability'].mean()
    
    gen_index.loc[gen_index['region'] == region, 
                  'change in variability'] = (gen_index.loc[gen_index['region'] == region, 
                                                            'Index variability'] - variability_base) / variability_base
    
    gen_index.loc[gen_index['region'] == region, 
                  'change in norm variability'] = (gen_index.loc[gen_index['region'] == region, 
                                                            'Normalized Index variability'] - norm_variability_base) / norm_variability_base

In [20]:
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.plotting import *
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.palettes import viridis

In [21]:
output_notebook()

Loading BokehJS ...

In [36]:
def weighted_percent(df, fuel, year):
    all_fuels = ['Coal', 'Natural Gas', 'Nuclear', 'Other', 'Other Renewables',
                 'Wind', 'Solar', 'Hydro']
    temp = df.loc[df['Year'] == year, all_fuels]
    temp['Total'] = temp.sum(axis=1)
    
    weighted_per = temp[fuel].sum() / temp['Total'].sum() * 100
    return weighted_per

In [37]:
weighted_percent(gen_index, 'Wind', 2005)

0.4207083600057916

In [33]:
gen_index.columns

Index([u'region', u'datetime', u'Coal', u'Hydro', u'Natural Gas', u'Nuclear',
       u'Other', u'Other Renewables', u'Solar', u'Wind', u'Year',
       u'monthly index (g/kWh)', u'Index variability',
       u'Normalized Index variability', u'Total gen', u'percent Coal',
       u'change in Coal', u'percent Natural Gas', u'change in Natural Gas',
       u'percent Other Renewables', u'change in Other Renewables',
       u'percent Nuclear', u'change in Nuclear', u'percent Other',
       u'change in Other', u'percent Solar', u'change in Solar',
       u'percent Wind', u'change in Wind', u'percent Hydro',
       u'change in Hydro', u'change in variability',
       u'change in norm variability'],
      dtype='object')

In [40]:
def region_bokeh_plot(df, fuel, x_axis, y_axis, x_range=None, y_range=None,
                     x_lower_lim=0, x_upper_lim=None):
    figs = {}
    if not x_upper_lim:
        x_upper_lim = df[x_axis].max()
        
    for region in df['region'].unique():
        temp = gen_index.loc[gen_index['region'] == region]
        max_change = temp[x_axis].max()
        
        if max_change <= x_upper_lim and max_change >= x_lower_lim:
            
            source = ColumnDataSource(data=dict(
            x=temp[x_axis],
            y=temp[y_axis],
            datetime=temp['datetime'],
            colors= viridis(len(temp))
            ))
            
            hover = HoverTool(tooltips=[
                                    ("(x,y)", "($x, $y)"),
                                    ("datetime", "@datetime{%F}")],
                             formatters={
                                     'datetime': 'datetime', # use 'datetime' formatter
                                        }
                             )
            hover.point_policy = "snap_to_data"

            fuel_2005 = weighted_percent(temp, fuel, 2005)
            fuel_2016 = weighted_percent(temp, fuel, 2016)
            title = region + ' {:.1f}% to {:.1f}% {}'.format(fuel_2005, 
                                                             fuel_2016, fuel)

            figs[region] = figure(title=title, tools=[hover], y_range=y_range, x_range=x_range,
                                 output_backend="webgl")
            figs[region].circle('x', 'y', source=source, color='colors', size=12, alpha=0.5)
            figs[region].xaxis.axis_label = x_axis
            figs[region].yaxis.axis_label = y_axis

    plots = figs.values()
    grid = gridplot(plots, ncols=4, plot_width=240, plot_height=240)

    show(grid, notebook_handle=True)

### Wind

In [44]:
region_bokeh_plot(gen_index, 'Wind', 'change in Wind', 
                  'change in norm variability', y_range=(-1, 7))

In [52]:
region_bokeh_plot(gen_index, 'Wind', 'percent Wind', 
                  'change in norm variability', 
                  y_range=(-1, 7), x_range=(0, 0.2))

### Natural gas

In [49]:
region_bokeh_plot(gen_index, 'Natural Gas', 'change in Natural Gas', 
                  'change in norm variability', y_range=(-1, 7))

In [56]:
region_bokeh_plot(gen_index, 'Natural Gas', 'percent Natural Gas', 
                  'change in norm variability',
                  y_range=(-1, 7), x_range=(0, 0.7))

### Coal

In [50]:
region_bokeh_plot(gen_index, 'Coal', 'change in Coal', 
                  'change in norm variability', y_range=(-1, 7))

In [58]:
region_bokeh_plot(gen_index, 'Coal', 'percent Coal', 
                  'change in norm variability',
                  y_range=(-1, 7), x_range=(0, 0.8))

### Solar
For some reason HICC doesn't have facility solar generation reported in 2016/17

In [47]:
region_bokeh_plot(gen_index, 'Solar', 'change in Solar', 
                  'change in norm variability', y_range=(-1, 7))

In [60]:
region_bokeh_plot(gen_index, 'Solar', 'percent Solar', 
                  'change in norm variability',
                  y_range=(-1, 7), x_range=(0, 0.05))

### Hydro
I'm not sure if the reduction in hydro is real - need to check data

In [48]:
region_bokeh_plot(gen_index, 'Hydro', 'change in Hydro', 
                  'change in norm variability', y_range=(-1, 7))